In [1]:
import os
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate

# ─── Configuration ─────────────────────────────────────────────────────────
FOLDS_DIR    = "cv_folds"
PLAYER_FEATS = ["PTS", "REB", "AST"]
OPP_FEATS    = ["opp_E_OFF_RATING", "opp_E_DEF_RATING", "opp_E_NET_RATING", "opp_REB_PCT"]
TARGET_COLS  = ["PTS", "REB", "AST"]
N_JOBS       = 4

# ─── Hyperparameters to try ─────────────────────────────────────────────────
hyperparams = [
    {"units": 16, "epochs": 10},
    {"units": 16, "epochs": 20},
    {"units": 32, "epochs": 10},
    {"units": 32, "epochs": 20},
]

# ─── Build dual-input LSTM ──────────────────────────────────────────────────
def make_dual_lstm(timesteps, n_feats, n_opp, n_outputs, units):
    # sequence input
    seq_in = Input(shape=(timesteps, n_feats), name="player_seq")
    # LSTM layer
    x = LSTM(units, name="lstm")(seq_in)

    # opponent stats input
    opp_in = Input(shape=(n_opp,), name="opp_stats")
    # combine sequence encoding + opponent stats
    m = Concatenate()([x, opp_in])

    # dense head
    h = Dense(16, activation="relu")(m)
    out = Dense(n_outputs)(h)

    model = Model([seq_in, opp_in], out)
    model.compile(optimizer="adam", loss="mse")
    return model

# ─── Sequence builders ─────────────────────────────────────────────────────
def build_train_sequences(df):
    X_seq, X_opp, Y = [], [], []
    for pid, gp in df.groupby("PLAYER_ID"):
        gp = gp.sort_values("GAME_DATE")
        p = gp[PLAYER_FEATS].values
        o = gp[OPP_FEATS].values
        for i in range(8, len(gp)):
            X_seq.append(p[i-8:i])
            X_opp.append(o[i])
            Y.append(p[i])
    return np.array(X_seq), np.array(X_opp), np.array(Y)

def build_test_sequences(train_df, test_df):
    X_seq, X_opp, Y = [], [], []
    for row in test_df.sort_values("GAME_DATE").itertuples():
        pid = row.PLAYER_ID
        hist = train_df[train_df.PLAYER_ID == pid].sort_values("GAME_DATE")
        if len(hist) < 8:
            continue
        p = hist[PLAYER_FEATS].values
        X_seq.append(p[-8:])
        o_vec = [getattr(row, f) for f in OPP_FEATS]
        X_opp.append(o_vec)
        Y.append([row.PTS, row.REB, row.AST])
    return np.array(X_seq), np.array(X_opp), np.array(Y)

# ─── Single-fold evaluation ─────────────────────────────────────────────────
def eval_lstm_fold(fold_idx, hp):
    tr = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_train.csv", parse_dates=["GAME_DATE"])
    te = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_test.csv",  parse_dates=["GAME_DATE"])

    X_tr_seq, X_tr_opp, y_tr = build_train_sequences(tr)
    X_te_seq, X_te_opp, y_te = build_test_sequences(tr, te)

    # Scale sequence features
    ns, ts, nf = X_tr_seq.shape
    s_scaler = StandardScaler().fit(X_tr_seq.reshape(-1, nf))
    X_tr_seq = s_scaler.transform(X_tr_seq.reshape(-1, nf)).reshape(ns, ts, nf)
    X_te_seq = s_scaler.transform(X_te_seq.reshape(-1, nf)).reshape(-1, ts, nf)

    # Scale opponent features
    o_scaler = StandardScaler().fit(X_tr_opp)
    X_tr_opp = o_scaler.transform(X_tr_opp)
    X_te_opp = o_scaler.transform(X_te_opp)

    # Train LSTM
    model = make_dual_lstm(ts, nf, len(OPP_FEATS), len(TARGET_COLS), hp["units"])
    model.fit([X_tr_seq, X_tr_opp], y_tr,
              epochs=hp["epochs"],
              batch_size=32,
              verbose=0)

    # Predict
    preds = model.predict([X_te_seq, X_te_opp], verbose=0)

    # Compute per-target metrics
    metrics = {}
    for idx, target in enumerate(TARGET_COLS):
        y_true = y_te[:, idx]
        y_pred = preds[:, idx]
        metrics[f"{target}_rmse"] = np.sqrt(mean_squared_error(y_true, y_pred))
        metrics[f"{target}_mae"]  = mean_absolute_error(y_true, y_pred)
        metrics[f"{target}_r2"]   = r2_score(y_true, y_pred)
    return metrics

# ─── Run evaluation ─────────────────────────────────────────────────────────
all_results = []
for hp in hyperparams:
    fold_metrics = Parallel(n_jobs=N_JOBS)(
        delayed(eval_lstm_fold)(fold, hp) for fold in range(1, 11)
    )
    df = pd.DataFrame(fold_metrics)
    df["fold"]   = np.arange(1, len(df)+1)
    df["units"]  = hp["units"]
    df["epochs"] = hp["epochs"]
    all_results.append(df)

# concatenate all hyperparameter runs
final_df = pd.concat(all_results, ignore_index=True)

# reorder columns
cols = ["units", "epochs", "fold"] + [c for c in final_df.columns if c not in ("units", "epochs", "fold")]
final_df = final_df[cols]

# display results
print(final_df)


2025-05-04 23:00:50.670061: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-05-04 23:00:50.670089: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-05-04 23:00:50.670092: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-05-04 23:00:50.670109: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-04 23:00:50.670117: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-05-04 23:00:50.687475: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-05-04 23:00:50.687500: I metal_plugin/src/device/metal_device.cc:296] syste

    units  epochs  fold   PTS_rmse    PTS_mae    PTS_r2  REB_rmse   REB_mae  \
0      16      10     1  17.227127  14.842088 -2.897520  5.987361  4.848170   
1      16      10     2   6.918549   5.486083  0.372780  3.364956  2.607477   
2      16      10     3   6.612031   5.325053  0.392006  2.802144  2.163897   
3      16      10     4   7.015875   5.496743  0.382637  3.070682  2.320252   
4      16      10     5   6.759828   5.327305  0.410791  2.814472  2.151874   
5      16      10     6   7.003202   5.491405  0.398077  2.783115  2.141559   
6      16      10     7   6.892547   5.477912  0.373620  2.866715  2.216912   
7      16      10     8   7.130734   5.423855  0.383737  2.767369  2.147396   
8      16      10     9   6.707790   5.268682  0.358091  2.685450  2.094074   
9      16      10    10   7.285038   5.724221  0.341868  2.920266  2.240102   
10     16      20     1  17.280699  14.895611 -2.921798  6.170724  5.078451   
11     16      20     2   6.806462   5.370709  0.392